## Inteligência de Dados: Mapeamento e Análises Relacionais

### Cenário

O projeto visa desenvolver um banco de dados relacional para o Departamento de Inteligência de Dados (ID) de um banco fictício, utilizando um modelo que incorpora três tabelas principais: DadosOrigem, FluxoDados e Analises. Este modelo é projetado para suportar todo o processo de extração, carregamento, transformação e análise de dados, permitindo que a equipe de dados gerencie e analise informações de maneira eficiente. Utilizarei PostgreSQL como sistema de gerenciamento de banco de dados, o que traz vantagens como suporte a transações complexas, escalabilidade superior e robustez no gerenciamento de dados. Além disso, a biblioteca Pandas facilitará a manipulação e análise dos dados, complementando as capacidades do PostgreSQL.

Para preencher as tabelas com dados fictícios, será empregada a biblioteca Faker, que criará informações realistas de maneira aleatória, assegurando a integridade dos testes e simulações. Os comandos mágicos do IPython-SQL serão utilizados no Jupyter Notebook para simplificar a interação com o banco de dados e a execução de consultas SQL. Além disso, a visualização do modelo de entidade-relacionamento (ER) será realizada por meio da biblioteca Graphviz, possibilitando uma representação clara e intuitiva das relações entre as tabelas e seus respectivos fluxos de dados.

### Estrutura do Banco de Dados

#### 1. Tabela: DadosOrigem
Esta tabela armazena informações sobre as diferentes fontes de dados que estão sendo utilizadas.

| Coluna           | Tipo           | Descrição                                       |
|------------------|----------------|-------------------------------------------------|
| id_origem        | SERIAL (PK)    | Identificador único da origem dos dados         |
| nome_origem      | VARCHAR(255)   | Nome da origem dos dados                        |
| tipo_dado        | VARCHAR(100)   | Tipo de dado (ex: transacional, log, etc.)      |
| volume           | INT            | Volume estimado de dados na origem              |
| latencia         | VARCHAR(50)    | Latência de atualização (ex: real-time, diário) |
| descricao        | TEXT           | Descrição detalhada da origem                   |


#### 2. Tabela: FluxoDados
Esta tabela representa os fluxos de dados desde a origem até o consumo final.

| Coluna           | Tipo           | Descrição                                       |
|------------------|----------------|-------------------------------------------------|
| id_fluxo         | SERIAL (PK)    | Identificador único do fluxo de dados           |
| id_origem        | INT (FK)       | Referência para a origem dos dados              |
| destino          | VARCHAR(255)   | Destino do fluxo (ex: analistas, sistemas)      |
| status           | VARCHAR(50)    | Status do fluxo (ex: ativo, inativo)            |
| data_criacao     | TIMESTAMP      | Data em que o fluxo foi criado                  |
| data_atualizacao | TIMESTAMP      | Data da última atualização do fluxo             |

#### 3. Tabela: Analises
Esta tabela armazena as análises realizadas com os dados, incluindo suas hipóteses e resultados.

| Coluna           | Tipo           | Descrição                                       |
|------------------|----------------|-------------------------------------------------|
| id_analise       | SERIAL (PK)    | Identificador único da análise                  |
| id_fluxo         | INT (FK)       | Referência para o fluxo de dados utilizado      |
| hipoteses        | TEXT           | Hipóteses levantadas para a análise             |
| resultado        | TEXT           | Resultados da análise                           |
| data_analise     | TIMESTAMP      | Data em que a análise foi realizada             |
| responsavel      | VARCHAR(255)   | Nome do responsável pela análise                |

### Bibliotecas utilizadas

In [ ]:
import os
import warnings
from datetime import datetime

import pandas as pd
from dotenv import load_dotenv
import psycopg2
from graphviz import Digraph
from IPython.display import Image, display

from generate_random_data import DbConfig, DataGenerator
from postgres_setup import PostgresConnector


In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
def diagrama_er():
    dot = Digraph(comment='Diagrama ER do Banco de Dados')
    dot.attr(rankdir='LR')  # Orientação da esquerda para direita
    
    dot.node('DadosOrigem', '''dados_origem
    ----
    + id_origem (PK)
    nome_origem
    tipo_dado
    volume
    latencia
    descricao''')
    
    dot.node('FluxoDados', '''fluxo_dados
    ----
    + id_fluxo (PK)
    # id_origem (FK)
    destino
    status
    data_criacao
    data_atualizacao''')
    
    dot.node('Analises', '''analises
    ----
    + id_analise (PK)
    # id_fluxo (FK)
    hipoteses
    resultado
    data_analise
    responsavel''')
    
    # Adiciona relacionamentos
    dot.edge('DadosOrigem', 'FluxoDados', '1:N')
    dot.edge('FluxoDados', 'Analises', '1:N')
    
    return dot

In [ ]:
diagrama = diagrama_er()
diagrama.render('diagrama_er', format='png', cleanup=True)
display(Image(filename='diagrama_er.png'))

In [ ]:
load_dotenv()

In [ ]:
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")

In [ ]:
db_params = {
    'dbname': 'smart_data_db',
    'user': db_user,
    'password': db_password,
    'host': 'localhost',
    'port': '5432'
}

In [ ]:
postgres = PostgresConnector(**db_params)

In [ ]:
postgres.create_database_tables()

In [ ]:
config = DbConfig(**db_params)

In [ ]:
with DataGenerator(config) as generator:
    df_origem, df_fluxo, df_analises = generator.gerar_e_inserir_dados()

In [ ]:
df_origem.head()

In [ ]:
df_fluxo.columns

In [ ]:
df_analises.shape

In [ ]:
info_df = postgres.get_table_info('dados_origem')
print(info_df)